In [5]:
import numpy as np
from sklearn.datasets import fetch_openml
from collections import Counter
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
import time
from collections import Counter


print("Fetching mnist_784 dataset... ")
X, y = fetch_openml('mnist_784', version=1, return_X_y=True, as_frame=False)
print("Fetching mnist_784 dataset completed")

X, y = X[:10000], y[:10000]  # Select first 10,000 images for processing
print(f"Using first 10,000 images: {X.shape[0]} training samples.")

print("Normalize pixel values to the range [0, 1] by dividing by 255")
X = X / 255.0  # Normalize pixel values to the range [0, 1] by dividing by 255
print("Convert labels to integers for compatibility with model training")
y = y.astype(int)  # Convert labels to integers for compatibility with model training


def compute_alpha_beta(X_class):
    # Initialize lists to store alpha and beta values for each feature
    alpha = []
    beta = []
    
    # Compute alpha and beta for each feature (pixel) for the given class
    for i in range(X_class.shape[1]):  # Loop over all features (784 features for MNIST)
        feature_data = X_class[:, i]  # Extract data for the ith feature (pixel)
        
        # Compute the mean and variance for this feature
        mean = np.mean(feature_data)  # Calculate the mean of the feature data
        variance = np.var(feature_data)  # Calculate the variance of the feature data
 
        # Avoid division by zero by checking if variance is zero
        if variance == 0:
            alpha_i = 1.0  # Set alpha to 1 if variance is zero
            beta_i = 1.0   # Set beta to 1 if variance is zero
        else:
            # Use the method of moments to estimate alpha and beta
            k = mean * (1 - mean) / variance  # Calculate k based on mean and variance
            alpha_i = mean * k  # Estimate alpha using the mean and k
            beta_i = (1 - mean) * k  # Estimate beta using the mean and k
        
        alpha.append(alpha_i)  # Append the computed alpha to the list
        beta.append(beta_i)    # Append the computed beta to the list
    
    return np.array(alpha), np.array(beta)  # Return the alpha and beta as numpy arrays
    

def predict_class(sample, alpha_dict, beta_dict, class_priors):
    max_log_prob = -np.inf  # Initialize maximum log probability to negative infinity
    predicted_class = -1  # Initialize predicted class to an invalid value
    epsilon = 1e-10  # Small constant to avoid log(0)
    
    # Ensure sample values are between epsilon and 1 - epsilon to avoid log(0) or log(1)
    sample = np.clip(sample, epsilon, 1 - epsilon)  
    
    # Iterate over each class label (0 to 9)
    for class_label in range(10):
        alpha = alpha_dict[class_label]  # Retrieve alpha parameter for the current class
        beta = beta_dict[class_label]  # Retrieve beta parameter for the current class
        
        # Compute log-likelihood for the current class using the beta-binomial distribution
        log_likelihoods = (alpha - 1) * np.log(sample) + (beta - 1) * np.log(1 - sample)
        
        # Calculate total log probability by summing log-likelihoods and adding the log of class prior
        total_log_prob = np.sum(log_likelihoods) + np.log(class_priors[class_label])
        
        # Update the predicted class if the current total log probability is greater than the maximum found
        if total_log_prob > max_log_prob:
            max_log_prob = total_log_prob  # Update maximum log probability
            predicted_class = class_label  # Update predicted class to the current class label

    return predicted_class  # Return the predicted class


print('Initialize KFold with 10 splits')
# Initialize KFold with 10 splits
kf = KFold(n_splits=10, shuffle=True, random_state=42)

# Store accuracy and time for each fold
fold_accuracies = []
fold_times = []

best_accuracy = 0  # To store the best accuracy
best_fold = -1  # To store the best fold index
best_alpha_mean = None  # To store the mean of alpha values for the best fold
best_beta_mean = None   # To store the mean of beta values for the best fold
best_train_data = None  # To store the training data of the best fold
best_test_data = None   # To store the test data of the best fold

# Perform 10-fold cross-validation
for fold_idx, (train_index, test_index) in enumerate(kf.split(X)):
    print(f"\nProcessing fold {fold_idx + 1}...")
    
    # Split the data into training and testing for this fold
    X_train, X_test = X[train_index], X[test_index]  # Select training and testing features
    y_train, y_test = y[train_index], y[test_index]  # Select training and testing labels
    
    # Recompute class priors for the training set
    class_counts = Counter(y_train)  # Count occurrences of each class in the training set
    total_samples = len(y_train)  # Total number of training samples
    class_priors = {class_label: count / total_samples for class_label, count in class_counts.items()}  # Calculate class priors
    
    # Compute alpha and beta for all classes based on the training set
    alpha_dict = {}  # Dictionary to store alpha values for each class
    beta_dict = {}   # Dictionary to store beta values for each class
    
    for class_label in range(10):  # Loop over all class labels (0 to 9)
        class_data = X_train[y_train == class_label]  # Extract data for the current class
        alpha, beta = compute_alpha_beta(class_data)  # Compute alpha and beta for the current class
        alpha_dict[class_label] = alpha  # Store alpha value
        beta_dict[class_label] = beta    # Store beta value
    
    # Start the timer for testing phase
    start_time = time.time()
    
    # Predict for all test samples
    predictions = []  # List to store predictions for the test set
    for test_sample in X_test:  # Iterate over each test sample
        predicted_class = predict_class(test_sample, alpha_dict, beta_dict, class_priors)  # Predict the class for the test sample
        predictions.append(predicted_class)  # Append the predicted class to the list
    
    # End the timer for testing phase
    end_time = time.time()
    
    # Calculate accuracy for this fold
    fold_accuracy = accuracy_score(y_test, predictions)  # Calculate accuracy by comparing predictions to true labels
    fold_accuracies.append(fold_accuracy)  # Store the accuracy for this fold
    fold_times.append(end_time - start_time)  # Store the time taken for testing this fold
    print(f"Accuracy for fold {fold_idx + 1}: {fold_accuracy * 100:.2f}%")
    print(f"Time taken for testing fold {fold_idx + 1}: {end_time - start_time:.4f} seconds")
    
    # Check if this fold has the best accuracy
    if fold_accuracy > best_accuracy:
        best_accuracy = fold_accuracy
        best_fold = fold_idx + 1  # Store the fold number (1-based index)
        best_alpha_mean = np.mean(list(alpha_dict.values()))  # Compute mean of alpha values
        best_beta_mean = np.mean(list(beta_dict.values()))    # Compute mean of beta values
        best_train_data = (X_train, y_train)  # Store the training data for the best fold
        best_test_data = (X_test, y_test)    # Store the test data for the best fold

# Display the best fold's configuration and accuracy
print(f"\nBest Accuracy: {best_accuracy * 100:.2f}% in fold {best_fold}")
print(f"Mean Alpha value for best fold: {best_alpha_mean}")
print(f"Mean Beta value for best fold: {best_beta_mean}")
print(f"Training Data (X_train, y_train) used in best fold: {best_train_data[0].shape, best_train_data[1].shape}")
print(f"Test Data (X_test, y_test) used in best fold: {best_test_data[0].shape, best_test_data[1].shape}")

# Calculate the overall mean accuracy and variance
mean_accuracy = np.mean(fold_accuracies)
variance_accuracy = np.var(fold_accuracies)
mean_time = np.mean(fold_times)
variance_time = np.var(fold_times)

# Print the summary statistics
print(f"\nMean Accuracy: {mean_accuracy * 100:.2f}%")
print(f"Variance in Accuracy: {variance_accuracy:.4f}")
print(f"Mean Time for Testing: {mean_time:.4f} seconds")
print(f"Variance in Time for Testing: {variance_time:.4f}")


Fetching mnist_784 dataset... 
Fetching mnist_784 dataset completed
Using first 10,000 images: 10000 training samples.
Normalize pixel values to the range [0, 1] by dividing by 255
Convert labels to integers for compatibility with model training
Initialize KFold with 10 splits

Processing fold 1...
Accuracy for fold 1: 34.50%
Time taken for testing fold 1: 2.5123 seconds

Processing fold 2...
Accuracy for fold 2: 37.20%
Time taken for testing fold 2: 2.1445 seconds

Processing fold 3...
Accuracy for fold 3: 32.40%
Time taken for testing fold 3: 2.0003 seconds

Processing fold 4...
Accuracy for fold 4: 33.70%
Time taken for testing fold 4: 2.0793 seconds

Processing fold 5...
Accuracy for fold 5: 38.10%
Time taken for testing fold 5: 2.0222 seconds

Processing fold 6...
Accuracy for fold 6: 37.00%
Time taken for testing fold 6: 2.2007 seconds

Processing fold 7...
Accuracy for fold 7: 36.20%
Time taken for testing fold 7: 2.0764 seconds

Processing fold 8...
Accuracy for fold 8: 29.30%
